# 1. Giới thiệu notebook
## Thí nghiệm: Training LBPH trên toàn bộ tập dữ liệu CelebA

Notebook này nhằm mục đích huấn luyện mô hình nhận dạng khuôn mặt LBPH trên toàn bộ tập dữ liệu CelebA để đánh giá khả năng mở rộng của nó trong các kịch bản nhận dạng khuôn mặt quy mô lớn.

# 2. Import và cài thư viện

In [2]:
!pip install opencv-contrib-python-headless psutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 28.8 MB/s eta 0:00:00:00:0100:01


In [3]:
import os
import cv2
import time
import psutil
import numpy as np
import pandas as pd
from tqdm import tqdm


# 3. Khai báo đường dẫn

In [4]:
IMG_DIR = (
    "/kaggle/input/celeba-dataset/"
    "img_align_celeba/img_align_celeba"
)

ID_FILE = (
    "/kaggle/input/identity-celeba/"
    "identity_CelebA.txt"
)


# 4. Load full identity file

In [5]:
df = pd.read_csv(
    ID_FILE,
    sep=" ",
    header=None,
    names=["image", "person_id"]
)

print("Total images:", len(df))
print("Total identities:", df["person_id"].nunique())


Total images: 202599
Total identities: 10177


# 5. Load ảnh (FULL DATASET, có đo RAM)

In [6]:
IMG_SIZE = (64, 64)

images = []
labels = []

process = psutil.Process(os.getpid())
start_mem = process.memory_info().rss / 1024**2
start_time = time.time()

In [7]:
for _, row in tqdm(df.iterrows(), total=len(df)):
    img_path = os.path.join(IMG_DIR, row["image"])
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        continue

    img = cv2.resize(img, IMG_SIZE)
    images.append(img)
    labels.append(row["person_id"])

100%|██████████| 202599/202599 [36:39<00:00, 92.11it/s]  


In [8]:
images = np.array(images)
labels = np.array(labels)

print("Loaded images:", images.shape)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Loaded images: (202599, 64, 64)


# 6. Encode label

In [9]:
unique_ids = np.unique(labels)
label_map = {pid: idx for idx, pid in enumerate(unique_ids)}
labels_enc = np.array([label_map[l] for l in labels])

print("Number of classes:", len(unique_ids))


Number of classes: 10177


# 7. Train LBPH + log time

In [10]:
lbph = cv2.face.LBPHFaceRecognizer_create(
    radius = 1,
    neighbors = 8,
    grid_x = 8,
    grid_y = 8
)

train_start = time.time()

try:
    lbph.train(images, labels_enc)
    train_success = True
except Exception as e:
    train_success = False
    error_msg = str(e)

train_time = time.time() - train_start


# 8. Save checkpoint + log

In [14]:
import json
end_mem = process.memory_info().rss / 1024**2
total_time = time.time() - start_time

log = {
    "train_success": train_success,
    "num_images": int(len(images)),
    "num_classes": int(len(unique_ids)),
    "train_time_sec": train_time,
    "total_runtime_sec": total_time,
    "memory_increase_MB": end_mem - start_mem
}

with open("training_log.json", "w") as f:
    json.dump(log, f, indent = 4)

if train_success:
    lbph.save("lbph_full_celeba.xml")

print(log)


{'train_success': True, 'num_images': 202599, 'num_classes': 10177, 'train_time_sec': 123.275563955307, 'total_runtime_sec': 3127.875587940216, 'memory_increase_MB': 13638.19921875}


# 9. Kết luận notebook train
## Quan sát quá trình huấn luyện

Quá trình huấn luyện cho thấy thời gian chạy và mức tiêu thụ bộ nhớ quá cao,
cho thấy LBPH không thể mở rộng quy mô đối với các tập dữ liệu có hơn 10.000 danh tính.

In [15]:
!ls /kaggle/working

lbph_full_celeba.xml  training_log.json


In [ ]:
import os
import zipfile

OUTPUT_FILES = [
    "training_log.json",
    "lbph_full_celeba.xml"
]

zip_name = "lbph_notebook1_output.zip"

with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as zipf:
    for file in OUTPUT_FILES:
        if os.path.exists(file):
            zipf.write(file)
            print(f"Added: {file}")
        else:
            print(f"Skipped (not found): {file}")

print(f"\nZip file created: {zip_name}")
